In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import networkx as nx

In [4]:
data_path = '/public/npatil4/networks/data/combined/'

In [5]:
file_year = data_path + 'data_2015_withNodes_year.txt'
data_year = pd.read_csv(file_year,
                        sep='|')

In [6]:
def data_limited(df):
    
    mask = (
        (data_year.pick_long.between(-75,-73).values) & 
        (data_year.pick_lat.between(40,41).values) &
        (data_year.drop_long.between(-75,-73).values) & 
        (data_year.drop_lat.between(40,41).values)
        )
    
    return(mask)

def data_limited2(df):
    
    mask = (
        (data_year.pick_long.between(-74.02,-73.92).values) & 
        (data_year.pick_lat.between(40.70,40.85).values) &
        (data_year.drop_long.between(-74.02,-73.92).values) & 
        (data_year.drop_lat.between(40.70,40.85).values)
        )
    
    return(mask)

##### Trips in manhattan

In [7]:
mask = (data_limited2(data_year))
print 'Routes : ',sum(mask)
print 'Trips : ',data_year.trips[mask].sum()

Routes :  11146000
Trips :  125169721


##### Graph Generation and total nodes

In [8]:
data_nwk =nx.DiGraph()
data_nwk = nx.from_pandas_dataframe(df = data_year[mask],
                                    source='start',
                                    target='stop',
                                    edge_attr=['trips'],
                                    create_using=data_nwk)

In [9]:
del data_year

In [ ]:
from censusgeocode import CensusGeocode
cg = CensusGeocode()

graph = data_nwk
graph_census_tracts = {}
graph_census_blocks ={}

node_list = graph.nodes()
census_data = {
                str(x):cg.coordinates( eval(x)[1], eval(x)[0] )
                for x in node_list
                }

In [12]:
for node in node_list:
    
    graph_census_tracts[node] = census_data[node][0]['Census Tracts']
    graph_census_blocks[node] = census_data[node][0]['2010 Census Blocks']

    

# Matching geo-coordinate to census tract

graph_census_tracts_2 = pd.DataFrame.from_dict(
                        { location:graph_census_tracts[location][0] 
                          for location in graph_census_tracts.keys()}).transpose()
graph_census_tracts_2.reset_index(inplace=True)

# Matching geo-coordinate to census Block

graph_census_blocks_2 = pd.DataFrame.from_dict(
                        { location:graph_census_blocks[location][0] 
                          for location in graph_census_blocks.keys()}).transpose()
graph_census_blocks_2.reset_index(inplace=True)



graph_census_tracts_2.to_csv('../data/graph_censusTract_full.txt',sep='|',index=False)
graph_census_blocks_2.to_csv('../data/graph_censusBlock_full.txt',sep='|',index=False)